# Анализ данных о продажах в маркетплейсе

Источник данных: https://www.kaggle.com/datasets/shreyanshverma27/online-sales-dataset-popular-marketplace-data/data

Исследование датасета с данными о транзакциях онлайн-продаж по различным категориям товаров. Каждая строка представляет собой отдельную транзакцию с подробной информацией, такой как идентификатор заказа, дата, категория, название продукта, проданное количество, цена за единицу, общая цена, регион и способ оплаты.

**Ход исследования:**

1. Работа с данными (общая информация, предобработка);
3. Исследовательский анализ данных:
    * Общая информация;
    * Продажи относительно категорий товаров и регионов;
    * Анализ продаж с течением времени;
    * ТОП-10 товаров каждой из категорий;
4. Выводы

In [2]:
# импорт всех необходимых библиотек

import pandas as pd

## Данные

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
df = pd.read_csv('/content/gdrive/MyDrive/analysis_git/online_sales.csv')
df

,Transaction ID,Date,Product Category,Product Name,Units Sold,Unit Price,Total Revenue,Region,Payment Method
0,10001,2024-01-01,Electronics,iPhone 14 Pro,2,999.99,1999.98,North America,Credit Card
1,10002,2024-01-02,Home Appliances,Dyson V11 Vacuum,1,499.99,499.99,Europe,PayPal
2,10003,2024-01-03,Clothing,Levi's 501 Jeans,3,69.99,209.97,Asia,Debit Card
3,10004,2024-01-04,Books,The Da Vinci Code,4,15.99,63.96,North America,Credit Card
4,10005,2024-01-05,Beauty Products,Neutrogena Skincare Set,1,89.99,89.99,Europe,PayPal
...,...,...,...,...,...,...,...,...,...
235,10236,2024-08-23,Home Appliances,Nespresso Vertuo Next Coffee and Espresso Maker,1,159.99,159.99,Europe,PayPal
236,10237,2024-08-24,Clothing,Nike Air Force 1 Sneakers,3,90.00,270.00,Asia,Debit Card
237,10238,2024-08-25,Books,The Handmaid's Tale by Margaret Atwood,3,10.99,32.97,North America,Credit Card
238,10239,2024-08-26,Beauty Products,Sunday Riley Luna Sleeping Night Oil,1,55.00,55.00,Europe,PayPal


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240 entries, 0 to 239
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Transaction ID    240 non-null    int64  
 1   Date              240 non-null    object 
 2   Product Category  240 non-null    object 
 3   Product Name      240 non-null    object 
 4   Units Sold        240 non-null    int64  
 5   Unit Price        240 non-null    float64
 6   Total Revenue     240 non-null    float64
 7   Region            240 non-null    object 
 8   Payment Method    240 non-null    object 
dtypes: float64(2), int64(2), object(5)
memory usage: 17.0+ KB


Согласно документации к данным:

* `Transaction ID` — уникальный идентификатор каждой транзакции;
* `Date` — дата транзакции;
* `Product Category` — категория товара (например, электроника, бытовая техника, Одежда, Книги, косметические товары, спортивные товары);
* `Product Name` — название товара;
* `Units Sold` — количество единиц товара, проданных в рамках транзакции;
* `Unit Price` — цена одной единицы товара;
* `Total Revenue` — общая выручка, полученная в результате транзакции (Unit Price * Units Sold)
* `Region` — географический регион, в котором была произведена транзакция (например, Северная Америка, Европа, Азия)
* `Payment Method` — способ оплаты (например, кредитная карта, PayPal, дебетовая карта)

В датасете хранится информация о 240 заказах в интернет-магазине. Явных пропусков не наблюдается.

In [7]:
# Проверка на полные дубликаты

print(f'Количество полных дубликатов: {df.duplicated().sum()}')

Количество полных дубликатов: 0


In [14]:
# приведение полей к lower case

df.columns = df.columns.str.replace(' ', '_').str.lower()

Избавимся от столбца с идентификаторами транзакций, так как они не несут в себе важной для анализа информации:

In [17]:
df = df.drop('transaction_id', axis=1)

Добавим недостающие для анализа данные: из столбца `date` вытащим информацию о дне недели и месяце.

In [21]:
# Преобразуем столбец 'date' в datetime
df['date'] = pd.to_datetime(df['date'])

# Получаем номер дня недели, где понедельник равен 1, а воскресенье равен 7
df['day_of_week'] = (df['date'].dt.dayofweek + 1) % 7
df.loc[df['day_of_week'] == 0, 'day_of_week'] = 7

# Добавляем столбец с номерами месяцев
df['month_number'] = df['date'].dt.month

In [23]:
df.sample(5)

,date,product_category,product_name,units_sold,unit_price,total_revenue,region,payment_method,day_of_week,month_number
33,2024-02-03,Books,1984 by George Orwell,4,19.99,79.96,North America,Credit Card,6,2
26,2024-01-27,Clothing,Adidas Ultraboost Shoes,2,179.99,359.98,Asia,Debit Card,6,1
113,2024-04-23,Sports,Fitbit Versa 3,3,229.95,689.85,Asia,Credit Card,2,4
12,2024-01-13,Electronics,Samsung Galaxy Tab S8,2,749.99,1499.98,North America,Credit Card,6,1
182,2024-07-01,Clothing,Levi's 511 Slim Fit Jeans,3,59.99,179.97,Asia,Debit Card,1,7


## Анализ данных

Ссылка на дэшборд `Yandex Datalens`: https://datalens.yandex/i53pgc4mnglw6

Далее будут описаны наблюдения о данных на основе графиков на этом дэшборде.

### Общая информация

Датасет содержит информацию о `240` транзакциях, каждая из которых совершалась в отдельный день. В данных нет транзакций, совершенных в один день.

В одном заказе может содержаться несколько единиц одного товара, всего в датасете хранится информация о продаже `518` товаров, суммарная выручка составляет `80 568` долларов. В среднем выручка с одного заказа составляет `336` долларов.


**Покупаемые товары разделены на шесть категорий:**
  * Электроника
  * Товары для дома
  * Книги
  * Спорт
  * Косметические средства
  * Одежда

**Заказы поступали из трех регионов:**
  * Азия
  * Северная Америка
  * Европа

**Выделено три способа оплаты:**
  * PayPal
  * Кредитная карта
  * Дебетовая карта


### Продажи относительно категорий товаров и регионов

Ожидаемо, что наиболее прибыльными оказались товары из категории `"Электроника"`, так как в среднем они стоят гораздо больше, чем товары других категорий. Хотя при этом товары из этой категории заказывают не так часто.

Наиболее часто заказываемыми товарами являются товары категории `"Одежда"`, в среднем в одном заказе содержится три единицы товара, в то время как товары категории `"Электроника"` обычно покупают по одному за один раз. При этом категория `"Одежда"` остается в четыре раза менее прибыльной чем `"Электроника"`.

Второй по частоте заказов категорией товаров являются `"Книги"` но в связи с их маленькой ценой они являются наименее прибыльными и занимают по этому параметру последнее место.

Наименее популярной является категория `"Косметические средства"`.



Что касается регионов, наибольшее число заказов совершают в `Северной Америке` - 45% всей выручки поступает оттуда.

При этом в Северной Америке зафиксированы только заказы `книг` и `электроники`. Все заказы электронных товаров приходятся именно на этот регион, поэтому доля выручки в нем самая высокая.


Доли суммарной выручки от регионов `Азия` и `Европа` почти равны (Азия - 27% и Европа - 26%).

В `Азии` заказывают только товары категорий `Одежда` и `Спорт`, а в `Европе` - товары из оставшихся категорий `Товары для дома` и `Косметические средства`.

### Анализ продаж с течением времени

Можно заметить, что с января по апрель средняя выручка была значительно выше чем с мая по август. Возможно здесь имеет место некоторая сезонность. По имеющимся данным нельзя утверждать о ней однозначно: для этого необходимо собрать исторические данные за последние несколько лет.

Наибольшее количество товаров заказывали в феврале и марте. Товары из категории `спорт` гораздо чаще покупают в "холодное" время года.

Что касается дней недели, наибольшая выручка зафиксирована по вторникам и пятницам, хотя по количеству заказанных товаров лидируют суббота и воскресенье.

Электронику и спортивные товары в среднем чаще заказывают в субботу.

### ТОП-10 товаров каждой из категорий

**`Электроника`**

1. iPhone 14 Pro
2. Samsung QLED 4K TV
3. Samsung Odyssey G9 Gaming Monitor
4. Samsung Galaxy Tab S8
5. Microsoft Surface Laptop 4
6. MacBook Pro 16-inch
7. HP Spectre x360 Laptop
8. Canon EOS R5 Camera
9. Apple MacBook Pro 16-inch
10. Apple MacBook Air

**`Косметические средства:`**

1. Tom Ford Black Orchid Perfume
2. Sunday Riley Good Genes
3. Neutrogena Skincare Set
4. Lancome La Vie Est Belle
5. La Mer Crème de la Mer Moisturizer
6. Kiehl's Midnight Recovery Concentrate
7. Estee Lauder Advanced Night Repair
8. Dyson Supersonic Hair Dryer
9. Charlotte Tilbury Magic Cream
10. Chanel No. 5 Perfume

**`Одежда`**

1. Ray-Ban Aviator Sunglasses
2. Patagonia Better Sweater
3. North Face Down Jacket
4. Nike Air Force 1 Sneakers
5. Nike Air Force 1
6. Lululemon Align Leggings
7. Adidas Ultraboost Shoes
8. Adidas Ultraboost Running Shoes
9. Adidas Originals Trefoil Hoodie
10. Adidas Originals Superstar Sneakers

**`Спорт`**

1. Polar Vantage V2
2. Peloton Bike
3. Manduka PRO Yoga Mat
4. GoPro HERO9 Black
5. Garmin Forerunner 945
6. Garmin Fenix 6X Pro
7. Garmin Edge 530
8. Fitbit Versa 3
9. Bowflex SelectTech 1090 Adjustable Dumbbells
10. Babolat Pure Drive Tennis Racket

**`Книги`**

1. Where the Crawdads Sing by Delia Owens
2. To Kill a Mockingbird by Harper Lee
3. The Silent Patient by Alex Michaelides
4. The Da Vinci Code
5. Salt, Fat, Acid, Heat by Samin Nosrat
6. Harry Potter and the Sorcerer's Stone
7. Educated by Tara Westover
8. Becoming by Michelle Obama
9. Atomic Habits by James Clear
10. 1984 by George Orwell

**`Товары для дома`**

1. Shark IQ Robot Vacuum
2. Roomba i7+
3. LG OLED TV
4. Eufy RoboVac 11S
5. Dyson V11 Vacuum
6. Dyson Supersonic Hair Dryer
7. De'Longhi Magnifica Espresso Machine
8. Breville Smart Grill
9. Blueair Classic 480i
10. Anova Precision Oven



## Выводы



* Стоит задуматься о расширении ассортимента товаров из категории `Электроника`, так как их продажа является наиболее прибыльной.

* Кроме того, магазину следует рассмотреть возможность расширения географии продаж электроники в регионы `Азии` и `Европы`. Для стимулирования интереса к электронной продукции рассматриваемого магазина в новых регионах, можно разработать специальные маркетинговые кампании, включающие выгодные предложения и акции, которые подчеркнут преимущества товаров этого магазина перед конкурентами и увеличат спрос на электронику в этих регионах.

* Самые низкие результаты продаж показывают товары категории `Косметические средства`, возможно стоит пересмотреть маркетинговую тактику относительно товаров этой группы для повышения продаж.

* Стоит обратить внимание на товары, входящие в ТОП-10 для каждой категории. Их запасы на складах стоит оптимизировать и направить маркетинговые стратегии в их сторону.

* С мая по август стоит обратить внимание на снижение продаж и направить маркетинговую стратегию на повышение покупок в это время года.

* Так как по выходным наблюдается рост количества покупок, возможно стоит устраивать выгодные предложения на отдельные товары в будние дни, чтобы увеличить количество заказов.

* Для исследования более точной сезонности необходимо собрать исторические данные за более длительный период.